# 33x POC results evaluation


## Setup and file download

Run the cells below to load the data and create feature definitions.

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
!pip install tensorflow_transform
import tensorflow_transform as tft


tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.6%}'.format;



In [0]:
# Connect GDrive to notebook
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load 33x adserver raw logs file from drive(copy and paste path):

## Data columns definition and summary

In [0]:
# Data columns definition (V2):
# columns = [
#     "day",
#     "hour",
#     "request_id",
#     "publisher_id",
#     "detected_domain",
#     "dsp_id",
#     "ad_size",
#     "product",
#     "device_type",
#     "country",
#     "tactic",
#     "pagecat",
#     "viewability",
#     "in_view_rate",
#     "time_in_view",
#     "fully_onscreen_in_view_rate",
#     "genderm",
#     "genderf",
#     "age18_34",
#     "age35_49",
#     "age50_plus",
#     "is_usa",
#     "is_buyer_matched",
#     "had_deals",
#     "response",
#     "response_0_score",
#     "response_1_score"
# ]

# columns V3
columns = [
    "is_winning_auction",
    "req_tstamp",
    "zone_id",
    "language",
    "country",
    "domain",
    "os",
    "browser",
    "geo_region",
    "geo_city",
    "geo_postal_code",
    "geo_dma_code",
    "geo_net_speed",
    "geo_continent",
    "pod_name",
    "deal_id",
    "sourcetype",
    "is_atf",
    "is_international",
    "flex_cv",
    "flex_gdpron",
    "flex_hasApp",
    "flex_hasVideo",
    "flex_iv",
    "web_user",
    "geo_source",
    "browser_engine",
    "is_whitelisted",
    "direct_on_page",
    "native_document_write",
    "endpoint",
    "zone_cpm_floor_price",
    "request_protocol",
    "auction_source",
    "foreign_publisher_id",
    "rtb_bidders_bidder_id",
    "rtb_bidders_result",
    "rtb_bidders_bid",
    "rtb_bidders_bid_cpm",
    "rtb_bidders_bid_0_score",
    "rtb_bidders_bid_1_score"
    
    
]


In [0]:
gz_path = r"/content/drive/My Drive/automl_oct_30_15hr_weight_45_prediction_000001_prediction-SOVRN_oct29_hrs14_1111063353_w45-2019-11-11T16_07_13.272Z_tables_14.csv";

data1 = pd.read_csv(gz_path, sep=",", header=0, na_values="", names=columns)
display.display(data1.describe());
gz_path2 = r"/content/drive/My Drive/automl_oct_30_15hr_weight_45_prediction_000001_prediction-SOVRN_oct29_hrs14_1111063353_w45-2019-11-11T16_07_13.272Z_tables_15.csv";

data2 = pd.read_csv(gz_path2, sep=",", header=0, 
                  na_values="", names=columns)
display.display(data2.describe());

#TODO: figure out how to make DataFrame to fetch data by batches limited by size and time



,req_tstamp,zone_id,geo_dma_code,flex_gdpron,flex_hasApp,flex_hasVideo,zone_cpm_floor_price,foreign_publisher_id,rtb_bidders_bidder_id,rtb_bidders_bid,rtb_bidders_bid_cpm,rtb_bidders_bid_0_score,rtb_bidders_bid_1_score
count,139661200.000000%,139661200.000000%,79898800.000000%,32278000.000000%,139661200.000000%,139661200.000000%,139661200.000000%,139661200.000000%,139661200.000000%,139661200.000000%,9034900.000000%,139661200.000000%,139661200.000000%
mean,157244941549279.687500%,57205592.510590%,61613.398199%,44.788091%,0.000000%,1.802147%,12.861282%,27060689618.813313%,2506.690835%,6.469155%,42.018632%,75.801410%,24.198588%
std,103688025.107477%,8641302.645732%,11107.222023%,49.727695%,0.000000%,13.302898%,36.761423%,237742576164.552368%,1922.095728%,24.598089%,96.475790%,23.849576%,23.849576%
min,157244760000600.000000%,12401600.000000%,50000.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,100.000000%,0.000000%,0.000100%,7.477326%,0.007371%
25%,157244851995200.000000%,57147200.000000%,51700.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,1000.000000%,0.000000%,3.856600%,57.867789%,3.131219%
50%,157244942215400.000000%,60702600.000000%,60000.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,2300.000000%,0.000000%,12.507000%,84.633103%,15.366896%
75%,157245031532975.000000%,62436600.000000%,68600.000000%,100.000000%,0.000000%,0.000000%,1.000000%,0.000000%,3600.000000%,0.000000%,41.730800%,96.868773%,42.132214%
max,157245119998900.000000%,65523700.000000%,88100.000000%,100.000000%,0.000000%,100.000000%,5010.000000%,2184131377200.000000%,8000.000000%,100.000000%,4430.509500%,99.992627%,92.522675%


,req_tstamp,zone_id,geo_dma_code,flex_gdpron,flex_hasApp,flex_hasVideo,zone_cpm_floor_price,foreign_publisher_id,rtb_bidders_bidder_id,rtb_bidders_bid,rtb_bidders_bid_cpm,rtb_bidders_bid_0_score,rtb_bidders_bid_1_score
count,139462900.000000%,139462900.000000%,79816700.000000%,32314900.000000%,139462900.000000%,139462900.000000%,139462900.000000%,139462900.000000%,139462900.000000%,139462900.000000%,9070000.000000%,139462900.000000%,139462900.000000%
mean,157244941629065.625000%,57209585.685154%,61638.005204%,44.753349%,0.000000%,1.792950%,12.744460%,27245963730.379478%,2507.241424%,6.503522%,42.337599%,75.789027%,24.210971%
std,103608340.126074%,8636475.251975%,11126.412577%,49.724042%,0.000000%,13.269531%,36.500836%,238560332639.156738%,1923.074941%,24.658808%,102.754468%,23.857409%,23.857409%
min,157244760001000.000000%,12401600.000000%,50000.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,100.000000%,0.000000%,0.000100%,7.575423%,0.007355%
25%,157244852295400.000000%,57147200.000000%,51700.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,1000.000000%,0.000000%,3.900000%,57.887739%,3.143517%
50%,157244942394400.000000%,60702600.000000%,60200.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,2300.000000%,0.000000%,12.554303%,84.624541%,15.375464%
75%,157245031447800.000000%,62436900.000000%,68600.000000%,100.000000%,0.000000%,0.000000%,1.000000%,0.000000%,3600.000000%,0.000000%,41.553600%,96.856481%,42.112255%
max,157245119999400.000000%,65523700.000000%,88100.000000%,100.000000%,0.000000%,100.000000%,5010.000000%,2183017795500.000000%,8000.000000%,100.000000%,9845.210000%,99.992645%,92.424572%


In [0]:
data = data1.append(data2);
display.display(data.describe());

,req_tstamp,zone_id,geo_dma_code,flex_gdpron,flex_hasApp,flex_hasVideo,zone_cpm_floor_price,foreign_publisher_id,rtb_bidders_bidder_id,rtb_bidders_bid,rtb_bidders_bid_cpm,rtb_bidders_bid_0_score,rtb_bidders_bid_1_score
count,279124100.000000%,279124100.000000%,159715500.000000%,64592900.000000%,279124100.000000%,279124100.000000%,279124100.000000%,279124100.000000%,279124100.000000%,279124100.000000%,18104900.000000%,279124100.000000%,279124100.000000%
mean,157244941589144.406250%,57207587.679423%,61625.695377%,44.770710%,0.000000%,1.797552%,12.802913%,27153260861.833679%,2506.965934%,6.486326%,42.178425%,75.795223%,24.204775%
std,103648207.690590%,8638889.684046%,11116.819837%,49.725832%,0.000000%,13.286235%,36.631494%,238151490274.258698%,1922.584725%,24.628447%,99.670520%,23.853487%,23.853487%
min,157244760000600.000000%,12401600.000000%,50000.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,100.000000%,0.000000%,0.000100%,7.477326%,0.007355%
25%,157244852142800.000000%,57147200.000000%,51700.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,1000.000000%,0.000000%,3.883800%,57.877374%,3.137347%
50%,157244942302900.000000%,60702600.000000%,60200.000000%,0.000000%,0.000000%,0.000000%,0.000000%,0.000000%,2300.000000%,0.000000%,12.527609%,84.628850%,15.371145%
75%,157245031489700.000000%,62436900.000000%,68600.000000%,100.000000%,0.000000%,0.000000%,1.000000%,0.000000%,3600.000000%,0.000000%,41.672300%,96.862656%,42.122623%
max,157245119999400.000000%,65523700.000000%,88100.000000%,100.000000%,0.000000%,100.000000%,5010.000000%,2184131377200.000000%,8000.000000%,100.000000%,9845.210000%,99.992645%,92.522675%


## Extra columns

In [0]:
selected_cols = data[
  [
      "req_tstamp",
      "rtb_bidders_bidder_id",
      "rtb_bidders_bid_0_score",
      "rtb_bidders_bid_1_score",
      "rtb_bidders_bid"
  ]].copy();
  
selected_cols["95_response_0"] = selected_cols["rtb_bidders_bid_0_score"]>0.91;
selected_cols["95_response_0_error"] = selected_cols.apply(lambda row: (row['rtb_bidders_bid_0_score'] > 0.91) & (row['rtb_bidders_bid'] == 1), axis=1);
selected_cols["99_response_0"] = selected_cols["rtb_bidders_bid_0_score"]>0.91;
selected_cols["99_response_0_error"] = selected_cols.apply(lambda row: (row['rtb_bidders_bid_0_score'] > 0.91) & (row['rtb_bidders_bid'] == 1), axis=1);



In [0]:
# Rename DSP into human readable
dsps = {
    27: '27',
  23: '23',
  43: '43',
  26: '26',
  1: '1',
  2: '2',
  16: '16',
  15: '15',
  10: '10',
  66: '66',
  36: '36',
  56: '56',
  3: '3',
  80: '80',
  58: '58',
  42: '42',
  76: '76',
  12: '12',
  49: '49',
  71: '71'
}

selected_cols['dsp'] = selected_cols.apply(lambda row: dsps[row['rtb_bidders_bidder_id']] if row['rtb_bidders_bidder_id'] in dsps else row['rtb_bidders_bidder_id'], axis=1);

In [0]:
i = 0
selected_cols['uuid'] = selected_cols.apply(lambda row: ++i, axis=1);

## Non-grouped stats

In [0]:


print("RESULTS");
display.display(selected_cols.head(2))
display.display(selected_cols.describe(percentiles=[.5,.9,.95,.99], include='all').transpose())

total = selected_cols['uuid'].count();
print("TOTAL REQUESTS");
display.display(total);
print("TOTAL 1");
total_1 = selected_cols[selected_cols['rtb_bidders_bid'] == 1]['uuid'].count();
display.display(total_1);
display.display(total_1 / float(total));
print("TOTAL 0");
total_0 = selected_cols[selected_cols['rtb_bidders_bid'] == 0]['uuid'].count();
display.display(total_0);
display.display(total_0 / float(total));
print("95% Threashold:");
print("===============");
print("Filtered count:");
response_0_count_95 = selected_cols[selected_cols['95_response_0'] == True]['uuid'].count();
display.display(response_0_count_95);
print("Filtered %:");
display.display(response_0_count_95 / float(total));
print("Errors count:");
response_0_count_errors_95 = selected_cols[selected_cols['95_response_0_error'] == True]['uuid'].count();
display.display(response_0_count_errors_95);
print("Erros %:");
display.display(response_0_count_errors_95 / float(total_1));


print("99% Threashold:");
print("===============");
print("Filtered count:");
response_0_count_99 = selected_cols[selected_cols['99_response_0'] == True]['uuid'].count();
display.display(response_0_count_99);
print("Filtered %:");
display.display(response_0_count_99 / float(total));
print("Errors count:");
response_0_count_errors_99 = selected_cols[selected_cols['99_response_0_error'] == True]['uuid'].count();
display.display(response_0_count_errors_99);
print("Erros %:");
display.display(response_0_count_errors_99 / float(total_1));





RESULTS


,req_tstamp,rtb_bidders_bidder_id,rtb_bidders_bid_0_score,rtb_bidders_bid_1_score,rtb_bidders_bid,95_response_0,95_response_0_error,99_response_0,99_response_0_error,dsp,uuid
0,1572448431134,26,50.393981%,49.606016%,0,False,False,False,False,26,0
1,1572450842199,10,77.878040%,22.121955%,0,False,False,False,False,10,0


,count,unique,top,freq,mean,std,min,50%,90%,95%,99%,max
req_tstamp,279124100.000000%,NaN,NaN,NaN,157244941589144.406250%,103648207.690590%,157244760000600.000000%,157244942302900.000000%,157245084539100.000000%,157245102233400.000000%,157245116432160.000000%,157245119999400.000000%
rtb_bidders_bidder_id,279124100.000000%,NaN,NaN,NaN,2506.965934%,1922.584725%,100.000000%,2300.000000%,5600.000000%,6600.000000%,8000.000000%,8000.000000%
rtb_bidders_bid_0_score,279124100.000000%,NaN,NaN,NaN,75.795223%,23.853487%,7.477326%,84.628850%,99.411500%,99.764484%,99.935573%,99.992645%
rtb_bidders_bid_1_score,279124100.000000%,NaN,NaN,NaN,24.204775%,23.853487%,0.007355%,15.371145%,62.208652%,70.258635%,80.581985%,92.522675%
rtb_bidders_bid,279124100.000000%,NaN,NaN,NaN,6.486326%,24.628447%,0.000000%,0.000000%,0.000000%,100.000000%,100.000000%,100.000000%
95_response_0,2791241,2,False,1666198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95_response_0_error,2791241,2,False,2790350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99_response_0,2791241,2,False,1666198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99_response_0_error,2791241,2,False,2790350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dsp,2791241,20,27,336744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


TOTAL REQUESTS


2791241

...

0.40306193553333447

Errors count:


891

Erros %:


0.004921319642748648

## Grouped stats

In [0]:
#Grouped by DSP ID

grouped_cols =  selected_cols.groupby('dsp')


print("TOTAL REQUESTS");
display.display(grouped_cols.size());
print("TOTAL 1");
total_1 = grouped_cols.agg((lambda g: g[g['rtb_bidders_bid'] == 1]['uuid'].count()))
display.display(total_1[['uuid']]);
display.display(
    grouped_cols.agg(
        (lambda g: g[g['rtb_bidders_bid'] == 1]['uuid'].count() / float(g['uuid'].count()))
    )[['req_tstamp']]
);
print("TOTAL 0");
display.display(grouped_cols.agg((lambda g: g[g['rtb_bidders_bid'] == 0]['uuid'].count()))[['uuid']]);
display.display(
    grouped_cols.agg(
        (lambda g: g[g['rtb_bidders_bid'] == 0]['uuid'].count() / float(g['uuid'].count()))
    )[['uuid']]
);



print("95% Threashold:");
print("===============");
print("Filtered count:");

display.display(
    grouped_cols.agg(
        (lambda g: g[g['95_response_0'] == True]['uuid'].count())
    )[['uuid']]
);

print("Filtered %:");
display.display(
    grouped_cols.agg(
        (lambda g: g[g['95_response_0'] == True]['uuid'].count() / float(g['uuid'].count()))
    )[['uuid']]
);
print("Errors count:");
display.display(
    grouped_cols.agg(
        (lambda g: g[g['95_response_0_error'] == True]['uuid'].count())
    )[['uuid']]
);
print("Erros %:");
display.display(
    (grouped_cols.agg(
        (lambda g: g[g['95_response_0_error'] == True]['uuid'].count())
    ) / total_1)[['uuid']]
);


print("99% Threashold:");
print("===============");
print("Filtered count:");
display.display(
    grouped_cols.agg(
        (lambda g: g[g['99_response_0'] == True]['uuid'].count())
    )[['uuid']]
);

print("Filtered %:");
display.display(
    grouped_cols.agg(
        (lambda g: g[g['99_response_0'] == True]['uuid'].count() / float(g['uuid'].count()))
    )[['uuid']]
);
print("Errors count:");
display.display(
    grouped_cols.agg(
        (lambda g: g[g['99_response_0_error'] == True]['req_tstamp'].count())
    )[['req_tstamp']]
);
print("Erros %:");
display.display(
    (grouped_cols.agg(
        (lambda g: g[g['99_response_0_error'] == True]['uuid'].count())
    ) / total_1)[['uuid']]
);



TOTAL REQUESTS


dsp
1     271256
10    128053
12     20354
15    149576
16    222485
2     267674
23    329956
26    281428
27    336744
3      49522
36    101058
42     32059
43    298271
49     13553
56     46437
58     42381
66    122059
71      5567
76     28692
80     44116
dtype: int64

TOTAL 1


,uuid
dsp,
1,2824
10,3179
12,240
15,366
16,1734
2,2615
23,66676
26,21152
27,23317


,req_tstamp
dsp,
1,1.041083%
10,2.482566%
12,1.179129%
15,0.244692%
16,0.779378%
2,0.976935%
23,20.207543%
26,7.515954%
27,6.924251%


...

,uuid
dsp,
1,55.431769%
10,40.320805%
12,57.295863%
15,82.454404%
16,60.555993%
2,54.938844%
23,10.204997%
26,13.914038%
27,40.001604%


Errors count:


,req_tstamp
dsp,
1,194
10,23
12,11
15,76
16,114
2,97
23,15
26,43
27,65


Erros %:


,uuid
dsp,
1,6.869688%
10,0.723498%
12,4.583333%
15,20.765027%
16,6.574394%
2,3.709369%
23,0.022497%
26,0.203290%
27,0.278767%
